In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataloader import *
import wandb

In [2]:
import pickle

# Open the file in binary mode for reading
with open('data/normalization_values.pkl', 'rb') as f:
    data = pickle.load(f)

# Now you can analyze the data
print(data)

{'S2_bands': {'B01': {'mean': 0.13021514, 'std': 0.017152175, 'min': 1e-04, 'max': 1.1213, 'p1': 0.1273, 'p99': 0.1074}, 'B02': {'mean': 0.1363337, 'std': 0.018509913, 'min': 1e-04, 'max': 1.8768, 'p1': 0.1366, 'p99': 0.1128}, 'B03': {'mean': 0.16427371, 'std': 0.02087248, 'min': 0.0411, 'max': 1.7888, 'p1': 0.1692, 'p99': 0.1364}, 'B04': {'mean': 0.13865142, 'std': 0.025569845, 'min': 0.0121, 'max': 1.7232, 'p1': 0.1445, 'p99': 0.1184}, 'B05': {'mean': 0.20296873, 'std': 0.028621713, 'min': 0.0672, 'max': 1.6344, 'p1': 0.2157, 'p99': 0.1591}, 'B06': {'mean': 0.38582557, 'std': 0.070499, 'min': 0.0758, 'max': 1.6699, 'p1': 0.3286, 'p99': 0.2766}, 'B07': {'mean': 0.4361872, 'std': 0.086211845, 'min': 0.0573, 'max': 1.6645, 'p1': 0.3621, 'p99': 0.2278}, 'B08': {'mean': 0.4448093, 'std': 0.08623231, 'min': 0.0737, 'max': 1.6976, 'p1': 0.3588, 'p99': 0.2122}, 'B8A': {'mean': 0.4580875, 'std': 0.08798952, 'min': 0.0772, 'max': 1.6709, 'p1': 0.3775, 'p99': 0.26}, 'B09': {'mean': 0.45806482, 

### creating the dataset mapping (train, test, val)

In [3]:
import h5py
import pandas as pd
import pickle
import os
from sklearn.model_selection import train_test_split

# Initialize an empty dictionary to store the data
data = {'train': [], 'val': [], 'test': []} 
path_h5 = '/scratch2/biomass_estimation/code/ml/data/data_no_outliers/'

all_tiles = []
# Iterate over all the h5 files
for fname in os.listdir(path_h5):
    if fname.endswith('.h5'):
        with h5py.File(os.path.join(path_h5, fname), 'r') as f:
            # Get the list of all tiles in the file
            all_tiles.extend(list(f.keys()))

train_tiles, test_and_val_tiles = train_test_split(all_tiles, test_size=0.35, random_state=42)
val_tile, test_tile = train_test_split(test_and_val_tiles, test_size=0.6, random_state=42)
data['val'].extend(val_tile)
data['test'].extend(test_tile)
data['train'].extend(train_tiles)

print("training tiles: ", len(data['train']))
print(data['train'])
print("validation tiles: ", len(data['val']))
print(data['val'])
print("testing tiles: ", len(data['test']))
print(data['test'])
# Pickle the DataFrame and save it to a file
with open('/scratch2/biomass_estimation/code/ml/data/mapping.pkl', 'wb') as f:
    pickle.dump(data, f)

training tiles:  45
['53UNQ', '52UFV', '53TPN', '53ULQ', '51UWQ', '53UNT', '52UFU', '51UXS', '52UFC', '51UVP', '53UMR', '50UQA', '51UVT', '51TWM', '53UNS', '51TVM', '52UFA', '54UUU', '54UVV', '54UVU', '51UVR', '51UYT', '52UGB', '51UYS', '53ULT', '51UXR', '53UPS', '54UUC', '51UUQ', '52UEA', '54UVA', '54UUA', '52UFB', '52UDA', '53TPL', '54UUV', '50TPT', '53UMT', '51UWS', '50TQT', '51UXQ', '51UWR', '53UPT', '51UVQ', '53UMS']
validation tiles:  10
['51UVS', '52UEV', '52UEC', '53UPP', '51TVN', '52UEB', '52UDC', '54UUB', '53ULR', '51UUP']
testing tiles:  15
['53TQN', '53UPQ', '51UWT', '52UCA', '51TWN', '51UUT', '51TVL', '50TPS', '52UGU', '51UWP', '53UMQ', '54UVC', '52UDB', '50UQB', '53UNR']


### defining loss and training args

In [4]:
class RMSE(nn.Module):
    """ 
        Weighted RMSE.
    """

    def __init__(self):
        super(RMSE, self).__init__()
        self.mse = torch.nn.MSELoss(reduction='none')
        
    def __call__(self, prediction, target, weights = 1):
        # prediction = prediction[:, 0]
        return torch.sqrt(torch.mean(weights * self.mse(prediction,target)))


class Args:
    def __init__(self):
        self.latlon = True
        self.bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
        self.bm = True
        self.patch_size = [15,15]
        self.norm_strat = 'pct'
        self.norm = False

args = Args()
fnames = ['data_nonan_0-5.h5', 'data_nonan_1-5.h5', 'data_nonan_2-5.h5', 'data_nonan_3-5.h5', 'data_nonan_4-5.h5']

### defining the SimpleFCN model

In [5]:
class SimpleFCN(nn.Module):
    def __init__(self,
                 in_features=18,
                 channel_dims = (16, 32, 64, 128),
                 num_outputs=1,
                 kernel_size=3,
                 stride=1):
        """
        A simple fully convolutional neural network.
        """
        super(SimpleFCN, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        layers = list()
        for i in range(len(channel_dims)):
            in_channels = in_features if i == 0 else channel_dims[i-1]
            layers.append(nn.Conv2d(in_channels=in_channels, 
                                    out_channels=channel_dims[i], 
                                    kernel_size=kernel_size, stride=stride, padding=1))
            layers.append(nn.BatchNorm2d(num_features=channel_dims[i]))
            layers.append(self.relu)
        print(layers)
        self.conv_layers = nn.Sequential(*layers)
        
        self.conv_output = nn.Conv2d(in_channels=channel_dims[-1], out_channels=num_outputs, kernel_size=1,
                                     stride=1, padding=0, bias=True)
        # self.fc = nn.Linear(15*15*num_outputs, 1)  # Fully connected layer to get a single output value

    def forward(self, x):
        x = self.conv_layers(x)
        # print(x.shape)
        x = self.conv_output(x)
        # x = x.flatten(start_dim=1)
        # predictions = self.fc(x)
        # return predictions.squeeze()  # Remove the extra dimension
        return x
    
model = SimpleFCN()

[Conv2d(18, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True)]


training simple FCN for 10 epochs with learning rate 0.001 (model3)

In [7]:

wandb.init()
model = SimpleFCN()
wandb.watch(model, log_freq=100)

if torch.cuda.is_available():
    model = model.cuda()
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

mode = 'train'
ds_training = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
trainloader = DataLoader(dataset = ds_training, batch_size = 512, shuffle = True, num_workers = 8)
mode = 'val'
ds_validation = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
validloader = DataLoader(dataset = ds_validation, batch_size = 512, shuffle = False, num_workers = 8)

min_valid_loss = float('inf')
# Training loop
for epoch in range(10):  # 10 epochs
    train_loss = 0.0
    model.train()
    i=0
    for inputs, targets in trainloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        # print("inputs.shape: ", inputs.shape)
        # print("targets.shape: ", targets.shape)
        # # # print(outputs)
        # print("outputs.shape: ", outputs.shape)
        # loss1 = criterion(outputs[:,:,7,7].squeeze(), targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        # print(loss.item())
        if i%20==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Training Loss: {train_loss / i}')
            wandb.log({'train_loss': train_loss / i})

    
    valid_loss = 0.0
    i=0
    model.eval()
    for inputs, targets in validloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        
        outputs = model(inputs)
        loss = criterion(outputs[:,:,7,7].squeeze(),targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        valid_loss += loss.item()
        if i%20==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Validation Loss: {valid_loss / i}')
            wandb.log({'valid_loss': valid_loss / i})
            np.save(f'training_predictions/outputs_epoch{epoch+1}_batch{i}.npy', outputs.detach().cpu().numpy())
        
 
    print(f'Epoch {epoch+1} Training Loss: {train_loss / len(trainloader)} Validation Loss: {valid_loss / len(validloader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss}--->{valid_loss}) Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model3.pth')


    print(f"Epoch {epoch+1} completed")

wandb.finish()

[Conv2d(18, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True)]
Epoch 1 	 Batch 20 	 Training Loss: 136.6802490234375
Epoch 1 	 Batch 40 	 Training Loss: 136.43809700012207
Epoch 1 	 Batch 60 	 Training Loss: 136.23353907267253
Epoch 1 	 Batch 80 	 Training Loss: 135.10922145843506
Epoch 1 	 Batch 100 	 Training Loss: 134.84507965087892
Epoch 1 	 Batch 120 	 Training Loss: 134.043644014

KeyboardInterrupt: 

### Defining bigger FCN

In [ ]:
class BiggerFCN(nn.Module):
    def __init__(self,
                 in_features=18,
                 channel_dims = (16, 32, 64, 128, 256, 512),
                 num_outputs=1,
                 kernel_size=3,
                 stride=1):
        """
        A simple fully convolutional neural network.
        """
        super(BiggerFCN, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        layers = list()
        for i in range(len(channel_dims)):
            in_channels = in_features if i == 0 else channel_dims[i-1]
            layers.append(nn.Conv2d(in_channels=in_channels, 
                                    out_channels=channel_dims[i], 
                                    kernel_size=kernel_size, stride=stride, padding=1))
            layers.append(nn.BatchNorm2d(num_features=channel_dims[i]))
            layers.append(self.relu)
        self.conv_layers = nn.Sequential(*layers)
        
        self.conv_output = nn.Conv2d(in_channels=channel_dims[-1], out_channels=num_outputs, kernel_size=1,
                                     stride=1, padding=0, bias=True)


    def forward(self, x):
        x = self.conv_layers(x)
        x = self.conv_output(x)
        return x
    
model = BiggerFCN()

In [ ]:
model = BiggerFCN()
if torch.cuda.is_available():
    model = model.cuda()
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

mode = 'train'
ds_training = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data/data_no_outliers', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
trainloader = DataLoader(dataset = ds_training, batch_size = 512, shuffle = True, num_workers = 8)
mode = 'val'
ds_validation = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data/data_no_outliers', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
validloader = DataLoader(dataset = ds_validation, batch_size = 512, shuffle = False, num_workers = 8)

min_valid_loss = float('inf')
# Training loop
for epoch in range(100):  # 100 epochs
    train_loss = 0.0
    model.train()
    i=0
    for inputs, targets in trainloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        # print("inputs.shape: ", inputs.shape)
        # print("targets.shape: ", targets.shape)
        # # # print(outputs)
        # print("outputs.shape: ", outputs.shape)
        # loss1 = criterion(outputs[:,:,7,7].squeeze(), targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        # print(loss.item())
        if i%20==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Training Loss: {train_loss / i}')

    
    valid_loss = 0.0
    i=0
    model.eval()
    for inputs, targets in validloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        
        outputs = model(inputs)
        loss = criterion(outputs[:,:,7,7].squeeze(),targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        valid_loss += loss.item()
        if i%20==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Validation Loss: {valid_loss / i}')
 
    print(f'Epoch {epoch+1} Training Loss: {train_loss / len(trainloader)} Validation Loss: {valid_loss / len(validloader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss}--->{valid_loss}) Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'bigger_model1.pth')


    print(f"Epoch {epoch+1} completed")
del model
del optimizer
del criterion
torch.cuda.empty_cache()

Epoch 1 	 Batch 20 	 Training Loss: 110.96225547790527
Epoch 1 	 Batch 40 	 Training Loss: 107.67209758758545
Epoch 1 	 Batch 60 	 Training Loss: 101.05285097757975
Epoch 1 	 Batch 80 	 Training Loss: 93.23420357704163
Epoch 1 	 Batch 100 	 Training Loss: 85.79349407196045
Epoch 1 	 Batch 120 	 Training Loss: 80.52298367818197
Epoch 1 	 Batch 140 	 Training Loss: 76.5239510672433
Epoch 1 	 Batch 160 	 Training Loss: 73.53724029064179
Epoch 1 	 Batch 180 	 Training Loss: 71.26314665476481
Epoch 1 	 Batch 200 	 Training Loss: 69.52043266296387
Epoch 1 	 Batch 220 	 Training Loss: 67.91062923778188
Epoch 1 	 Batch 240 	 Training Loss: 66.49623942375183
Epoch 1 	 Batch 260 	 Training Loss: 65.39369429074802
Epoch 1 	 Batch 280 	 Training Loss: 64.4995620727539
Epoch 1 	 Batch 300 	 Training Loss: 63.679258804321286
Epoch 1 	 Batch 320 	 Training Loss: 62.976606941223146
Epoch 1 	 Batch 340 	 Training Loss: 62.33444068011116
Epoch 1 	 Batch 360 	 Training Loss: 61.76926401986016
Epoch 1 	 B

### Defining bigger FCN pyramid

In [ ]:
class BiggerFCN_pyramid(nn.Module):
    def __init__(self,
                 in_features=18,
                 channel_dims = (32, 64, 128, 256, 512, 1024, 512, 256, 128, 64),
                 num_outputs=1,
                 kernel_size=3,
                 stride=1):
        """
        A simple fully convolutional neural network.
        """
        super(BiggerFCN_pyramid, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        layers = list()
        for i in range(len(channel_dims)):
            in_channels = in_features if i == 0 else channel_dims[i-1]
            layers.append(nn.Conv2d(in_channels=in_channels, 
                                    out_channels=channel_dims[i], 
                                    kernel_size=kernel_size, stride=stride, padding=1))
            layers.append(nn.BatchNorm2d(num_features=channel_dims[i]))
            layers.append(self.relu)
        self.conv_layers = nn.Sequential(*layers)
        
        self.conv_output = nn.Conv2d(in_channels=channel_dims[-1], out_channels=num_outputs, kernel_size=1,
                                     stride=1, padding=0, bias=True)


    def forward(self, x):
        x = self.conv_layers(x)
        x = self.conv_output(x)
        return x

In [ ]:
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
# del model
# del optimizer
# del criterion
# del trainloader
# del validloader
# torch.cuda.empty_cache()
### Defining bigger FCN

    
model = BiggerFCN_pyramid()
if torch.cuda.is_available():
    model = model.cuda()
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

mode = 'train'
ds_training = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data/data_no_outliers', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
trainloader = DataLoader(dataset = ds_training, batch_size = 128, shuffle = True, num_workers = 8)
# trainloader = DataLoader(dataset = ds_training, batch_size = 512, shuffle = True, num_workers = 8)
mode = 'val'
ds_validation = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data/data_no_outliers', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
# validloader = DataLoader(dataset = ds_validation, batch_size = 512, shuffle = False, num_workers = 8)
validloader = DataLoader(dataset = ds_validation, batch_size = 128, shuffle = False, num_workers = 8)

min_valid_loss = float('inf')
# Training loop
for epoch in range(100):  # 10 epochs
    train_loss = 0.0
    model.train()
    i=0
    for inputs, targets in trainloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        # print("inputs.shape: ", inputs.shape)
        # print("targets.shape: ", targets.shape)
        # # # print(outputs)
        # print("outputs.shape: ", outputs.shape)
        # loss1 = criterion(outputs[:,:,7,7].squeeze(), targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        # print(loss.item())
        if i%50==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Training Loss: {train_loss / i}')

    
    valid_loss = 0.0
    i=0
    model.eval()
    for inputs, targets in validloader:
        i+=1
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        
        outputs = model(inputs)
        loss = criterion(outputs[:,:,7,7].squeeze(),targets)
        loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
        valid_loss += loss.item()
        if i%50==0:
            print(f'Epoch {epoch+1} \t Batch {i} \t Validation Loss: {valid_loss / i}')
 
    print(f'Epoch {epoch+1} Training Loss: {train_loss / len(trainloader)} Validation Loss: {valid_loss / len(validloader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss}--->{valid_loss}) Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'bigger_model_pyramid.pth')


    print(f"Epoch {epoch+1} completed")

Epoch 1 	 Batch 50 	 Training Loss: 93.40515243530274
Epoch 1 	 Batch 100 	 Training Loss: 74.78540946960449
Epoch 1 	 Batch 150 	 Training Loss: 68.18138196309407
Epoch 1 	 Batch 200 	 Training Loss: 64.18197254180909
Epoch 1 	 Batch 250 	 Training Loss: 61.46015383911133
Epoch 1 	 Batch 300 	 Training Loss: 59.868708305358886
Epoch 1 	 Batch 350 	 Training Loss: 58.70942961556571
Epoch 1 	 Batch 400 	 Training Loss: 58.04661428451538
Epoch 1 	 Batch 450 	 Training Loss: 57.37459713406033
Epoch 1 	 Batch 500 	 Training Loss: 56.78074750518799
Epoch 1 	 Batch 550 	 Training Loss: 56.322076644897464
Epoch 1 	 Batch 600 	 Training Loss: 55.92796649297078
Epoch 1 	 Batch 650 	 Training Loss: 55.68372137803298
Epoch 1 	 Batch 700 	 Training Loss: 55.35711478642055
Epoch 1 	 Batch 750 	 Training Loss: 55.001108912150066
Epoch 1 	 Batch 800 	 Training Loss: 54.70549154281616
Epoch 1 	 Batch 850 	 Training Loss: 54.48975201775046
Epoch 1 	 Batch 900 	 Training Loss: 54.254909400939944
Epoch 1

### testing models

In [ ]:
# # Load the model
# model = BiggerFCN()
# model.load_state_dict(torch.load('bigger_model1.pth'))
# if torch.cuda.is_available():
#     model = model.cuda()

# # Set mode to 'test'
# mode = 'test'
# ds_testing = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
# testloader = DataLoader(dataset = ds_testing, batch_size = 512, shuffle = False, num_workers = 8)

# # Testing loop
# test_loss = 0.0
# model.eval()
# i = 0
# for inputs, targets in testloader:
#     i += 1
#     if torch.cuda.is_available():
#         inputs, targets = inputs.cuda(), targets.cuda()

#     outputs = model(inputs)
#     loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
#     test_loss += loss.item()
#     if i % 20 == 0:
#         print(f'Batch {i} \t Testing Loss: {test_loss / i}')

# print(f'Testing Loss bigger model: {test_loss / len(testloader)}')

torch.cuda.empty_cache()
model = BiggerFCN_pyramid()
# Load the model
model.load_state_dict(torch.load('bigger_model_pyramid.pth'))
if torch.cuda.is_available():
    model = model.cuda()

mode = 'test'
ds_testing = GEDIDataset({'h5':'/scratch2/biomass_estimation/code/ml/data/data_no_outliers', 'norm': '/scratch2/biomass_estimation/code/ml/data', 'map': '/scratch2/biomass_estimation/code/ml/data/'}, fnames = fnames, chunk_size = 1, mode = mode, args = args)
testloader = DataLoader(dataset = ds_testing, batch_size = 256, shuffle = False, num_workers = 8)

# Testing loop
test_loss = 0.0
model.eval()
i = 0
for inputs, targets in testloader:
    i += 1
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    loss = RMSE()(outputs[:,:,7,7].squeeze(), targets)
    test_loss += loss.item()
    if i % 20 == 0:
        print(f'Batch {i} \t Testing Loss: {test_loss / i}')

print(f'Testing Loss bigger model pyramid: {test_loss / len(testloader)}')

